In [99]:
import pandas as pd
import csv
import shlex

# Import Datasets

In [100]:
benchmark_path = '/home/ubuntu/shared/meg_shared_scripts/meg-kb/data/indeed-benchmark/benchmark.csv'
benchmark_evidence_path = '/home/ubuntu/shared/meg_shared_scripts/meg-kb/data/indeed-benchmark/benchmark_evidence.csv'
dataset_path = '/home/ubuntu/users/nikita/data/indeed/indeedQA/question_answers.csv'
company_path = '/home/ubuntu/users/nikita/data/indeed/indeedQA/fccid-companyName.csv'
corpus_path = '/home/ubuntu/users/nikita/src/openie6/dataset/indeed/corpus.txt'

In [101]:
df_bench = pd.read_csv(benchmark_path)
df_dataset = pd.read_csv(dataset_path) 
df_dataset = df_dataset[df_dataset['answerContent'].notna()]
df_company = pd.read_csv(company_path)

In [102]:
#df_bench.head()

In [103]:
df_merged_dataset = df_dataset.merge(df_company, how='inner', on='fccompanyId')

In [104]:
#df_merged_dataset.head()
# print(df_merged_dataset.loc[[320518]])

# Map Concept to Instance

In [105]:
lines = []

with open(benchmark_path) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_no = 1
    for row in csv_reader:
        line = row
        if line_no == 1:
            line_no += 1
            line.append('Evidence')
            lines.append(line)
            continue
        if row[0].lower() == 'company':
            answers = shlex.split(row[8][1:-1]) #row[8][1:-1].split(',')
            companies = []
            for answer in answers:
                result = df_merged_dataset.loc[df_merged_dataset['answerContent'].str.contains(answer), 'companyName']
                if len(result) != 0:
                    #print(result.iloc[0])
                    companies.append(result.iloc[0])
                else:
                    companies.append("")
            if len(companies) != 0:
                line.append(companies)
            else:
                line.append([])
        else:        
            line.append([])
        
        lines.append(line)
        line_no += 1
with open(benchmark_evidence_path, mode='w') as _file:
    _writer = csv.writer(_file, delimiter=',')
    for line in lines:
        print(line)
        _writer.writerow(line)

['n_head', 'n_tail', 'rel_phrases', 'most_freq_head', 'most_freq_tail', 'avg_freq', 'avg_score', 'in_seed', 'sentences', 'match_type', 'keep_in_benchmark', 'relation_name', 'n_head_category', 'n_tail_category', 'type', 'use_as_template', 'Evidence']
['company', '12 hour shifts', "['have']", 'company', '12 hour shifts then', '550', '0.8', '0', '["Where I work they have first shift I think 6am-4pm and second shift 4pm-2am both shifts are Monday-Friday and they have a 6pm-2am shift also Monday-Friday then they have 12 hour shifts and that\'s more the weekends"]', 'head', '1', 'has_shifts', 'company', 'shifts', 'fact', '0', ['']]
['walmart', '12 hour shifts', "['doesnt allow', 'did not offer']", 'walmart', 'for 12 hour shifts', '50', '0.96', '0', "['Walmart did not offer me any 12 hour shifts', 'Walmart doesnt allow for 12 hour shifts and are moving to a more standard hours shifts like 4am - 1pm, 1 pm-11 pm, as a department manager you can work 7am-4pm, and as an overnight person 10pm to 7